In [14]:
# Importing necessary libraries
import pandas as pd
import hashlib

## Dataset

- This dataset contain information in a Question Answering (QA) format from a diversified portofolio of well-known large companies with creat influence in their information.
- This QA records were created using the 2023-10-K annual financial reports.
- The selected dataset is a publicly available dataset hosted on Hugging Face Datasets page.
- You can find the dataset here: [asset-investment-financial-dataset](https://huggingface.co/datasets/shashankyadav03/asset-investment-financial-dataset)

### Understanding the dataset

In [2]:
# Loading dataset from Hugging Face
df = pd.read_parquet("hf://datasets/shashankyadav03/asset-investment-financial-dataset/data/train-00000-of-00001.parquet")
# Display the first rows of Dataset
df.head()

,question,answer,context,ticker,filing
0,What area did NVIDIA initially focus on before...,NVIDIA initially focused on PC graphics.,"Since our original focus on PC graphics, we ha...",NVDA,2023_10K
1,What are some of the recent applications of GP...,Recent applications of GPU-powered deep learni...,Some of the most recent applications of GPU-po...,NVDA,2023_10K
2,What significant invention did NVIDIA create i...,NVIDIA invented the GPU in 1999.,Our invention of the GPU in 1999 defined moder...,NVDA,2023_10K
3,How does NVIDIA's platform strategy contribute...,NVIDIA's platform strategy brings together har...,"NVIDIA has a platform strategy, bringing toget...",NVDA,2023_10K
4,What does NVIDIA's CUDA programming model enable?,NVIDIA's CUDA programming model opened the par...,With our introduction of the CUDA programming ...,NVDA,2023_10K


In [3]:
# See the total number of records
print('Number of records in the dataset: {} records \n'.format(df.shape[0]))
# Examine the different companies present in the dataset
print('''This dataset contains {} unique companies with tickers: 
{} \n'''.format(len(df.ticker.unique()),df.ticker.unique()))
# Examine how may different reports where used
print('{} unique financial Reports: {}'.format(len(df.filing.unique()),df.filing.unique()))

Number of records in the dataset: 7887 records 

This dataset contains 70 unique companies with tickers: 
['NVDA' 'AAPL' 'TSLA' 'LULU' 'PG' 'COST' 'ABNB' 'MSFT' 'BRK-A' 'META'
 'AXP' 'PTON' 'SBUX' 'NKE' 'PLTR' 'AMZN' 'NFLX' 'GOOGL' 'ABBV' 'V' 'GME'
 'AMC' 'CRM' 'LLY' 'AVGO' 'UNH' 'JNJ' 'HD' 'WMT' 'AMD' 'CVX' 'BAC' 'KO'
 'T' 'AZO' 'CAT' 'SCHW' 'CMG' 'CB' 'CMCSA' 'CVS' 'DVA' 'DAL' 'DLTR' 'EBAY'
 'EA' 'ENPH' 'EFX' 'ETSY' 'FDX' 'F' 'GRMN' 'GIS' 'GM' 'GILD' 'GS' 'HAS'
 'HSY' 'HPE' 'HLT' 'HPQ' 'HUM' 'IBM' 'ICE' 'INTU' 'IRM' 'JPM' 'KR' 'LVS'
 'UNKNOWN'] 

2 unique financial Reports: ['2023_10K' 'N/A']


In [4]:
# Examining how many QA records each company contains
print(df.ticker.value_counts())

UNKNOWN    887
JNJ        200
F          100
FDX        100
ETSY       100
          ... 
CVX        100
BAC        100
KO         100
T          100
CAT        100
Name: ticker, Length: 70, dtype: int64


In [5]:
# Examining the records with NA filing
print(df[df['filing'] == 'N/A'].ticker.unique())

['UNKNOWN']


In [6]:
# Examining Text in each text field
df['word_count_question'] = df['question'].apply(lambda x: len(str(x).split(' ')))
df['word_count_answer'] = df['answer'].apply(lambda x: len(str(x).split(' ')))
df['word_count_context'] = df['context'].apply(lambda x: len(str(x).split(' ')))

In [7]:
# For each question in dataset
print('''For Question column we have the below descriptive statistics: 
{} \n'''.format(df['word_count_question'].describe()))
# For each answer in dataset
print('''For Answer column we have the below descriptive statistics: 
{} \n'''.format(df['word_count_answer'].describe()))
# For each context in dataset
print('''For Context column we have the below descriptive statistics: 
{} \n'''.format(df['word_count_context'].describe()))

For Question column we have the below descriptive statistics: 
count    7887.000000
mean       14.224040
std         4.345017
min         1.000000
25%        11.000000
50%        14.000000
75%        17.000000
max        36.000000
Name: word_count_question, dtype: float64 

For Answer column we have the below descriptive statistics: 
count    7887.000000
mean       21.702548
std        16.338878
min         1.000000
25%         7.000000
50%        20.000000
75%        33.000000
max       124.000000
Name: word_count_answer, dtype: float64 

For Context column we have the below descriptive statistics: 
count    7887.000000
mean       33.721314
std        23.548573
min         1.000000
25%        20.000000
50%        29.000000
75%        40.000000
max       787.000000
Name: word_count_context, dtype: float64 



### Cleaning and Preparation of Dataset Steps

- Remove records with UNKOWN ticker
- Examine for duplicates - Mostly for JNJ
- Create a column with the Name of the company - Information will be generated from chat GPT-4o
- Create an unique id for each record
- Store the dataset for further use

In [8]:
# Removing the Unkown ticker records
df = df.drop(df[df['ticker'] == 'UNKNOWN'].index)
df.reset_index(inplace=True)

In [9]:
# Examine for duplicates
print('Number of duplicated records:',sum(df.duplicated()))

Number of duplicated records: 0


Chat GPT was used to generate this information with the below prompt

Please create a dictionary that can be used to converted into a Pandas data frame containing the following 2 columns:
1. The below list of Tickers:
['NVDA' 'AAPL' 'TSLA' 'LULU' 'PG' 'COST' 'ABNB' 'MSFT' 'BRK-A' 'META'
 'AXP' 'PTON' 'SBUX' 'NKE' 'PLTR' 'AMZN' 'NFLX' 'GOOGL' 'ABBV' 'V' 'GME'
 'AMC' 'CRM' 'LLY' 'AVGO' 'UNH' 'JNJ' 'HD' 'WMT' 'AMD' 'CVX' 'BAC' 'KO'
 'T' 'AZO' 'CAT' 'SCHW' 'CMG' 'CB' 'CMCSA' 'CVS' 'DVA' 'DAL' 'DLTR' 'EBAY'
 'EA' 'ENPH' 'EFX' 'ETSY' 'FDX' 'F' 'GRMN' 'GIS' 'GM' 'GILD' 'GS' 'HAS'
 'HSY' 'HPE' 'HLT' 'HPQ' 'HUM' 'IBM' 'ICE' 'INTU' 'IRM' 'JPM' 'KR' 'LVS']
2. The actual name of the company from each Ticker

In the next cell is the generated response.

In [10]:
# Genereated response from Chat GPT
data = {
    'ticker': ['NVDA', 'AAPL', 'TSLA', 'LULU', 'PG', 'COST', 'ABNB', 'MSFT', 'BRK-A', 'META',
               'AXP', 'PTON', 'SBUX', 'NKE', 'PLTR', 'AMZN', 'NFLX', 'GOOGL', 'ABBV', 'V',
               'GME', 'AMC', 'CRM', 'LLY', 'AVGO', 'UNH', 'JNJ', 'HD', 'WMT', 'AMD', 'CVX',
               'BAC', 'KO', 'T', 'AZO', 'CAT', 'SCHW', 'CMG', 'CB', 'CMCSA', 'CVS', 'DVA',
               'DAL', 'DLTR', 'EBAY', 'EA', 'ENPH', 'EFX', 'ETSY', 'FDX', 'F', 'GRMN',
               'GIS', 'GM', 'GILD', 'GS', 'HAS', 'HSY', 'HPE', 'HLT', 'HPQ', 'HUM', 'IBM',
               'ICE', 'INTU', 'IRM', 'JPM', 'KR', 'LVS'],
    'company': ['Nvidia Corporation', 'Apple Inc.', 'Tesla, Inc.', 'Lululemon Athletica Inc.', 
                     'Procter & Gamble Co.', 'Costco Wholesale Corporation', 'Airbnb, Inc.', 
                     'Microsoft Corporation', 'Berkshire Hathaway Inc.', 'Meta Platforms, Inc.',
                     'American Express Company', 'Peloton Interactive, Inc.', 'Starbucks Corporation', 
                     'Nike, Inc.', 'Palantir Technologies Inc.', 'Amazon.com, Inc.', 'Netflix, Inc.', 
                     'Alphabet Inc.', 'AbbVie Inc.', 'Visa Inc.', 'GameStop Corp.', 'AMC Entertainment Holdings, Inc.', 
                     'Salesforce, Inc.', 'Eli Lilly and Co.', 'Broadcom Inc.', 'UnitedHealth Group Incorporated', 
                     'Johnson & Johnson', 'The Home Depot, Inc.', 'Walmart Inc.', 'Advanced Micro Devices, Inc.', 
                     'Chevron Corporation', 'Bank of America Corporation', 'The Coca-Cola Company', 
                     'AT&T Inc.', 'AutoZone, Inc.', 'Caterpillar Inc.', 'Charles Schwab Corporation', 
                     'Chipotle Mexican Grill, Inc.', 'Chubb Limited', 'Comcast Corporation', 'CVS Health Corporation', 
                     'DaVita Inc.', 'Delta Air Lines, Inc.', 'Dollar Tree, Inc.', 'eBay Inc.', 
                     'Electronic Arts Inc.', 'Enphase Energy, Inc.', 'Equifax Inc.', 'Etsy, Inc.', 
                     'FedEx Corporation', 'Ford Motor Company', 'Garmin Ltd.', 'General Mills, Inc.', 
                     'General Motors Company', 'Gilead Sciences, Inc.', 'The Goldman Sachs Group, Inc.', 
                     'Hasbro, Inc.', 'The Hershey Company', 'Hewlett Packard Enterprise Co.', 
                     'Hilton Worldwide Holdings Inc.', 'HP Inc.', 'Humana Inc.', 'International Business Machines Corporation', 
                     'Intercontinental Exchange, Inc.', 'Intuit Inc.', 'Iron Mountain Incorporated', 
                     'JPMorgan Chase & Co.', 'Kroger Co.', 'Las Vegas Sands Corp.']
}

# Convert this as a dataframe
df_names = pd.DataFrame(data)
# View the first results
df_names.head(10)

,ticker,company
0,NVDA,Nvidia Corporation
1,AAPL,Apple Inc.
2,TSLA,"Tesla, Inc."
3,LULU,Lululemon Athletica Inc.
4,PG,Procter & Gamble Co.
5,COST,Costco Wholesale Corporation
6,ABNB,"Airbnb, Inc."
7,MSFT,Microsoft Corporation
8,BRK-A,Berkshire Hathaway Inc.
9,META,"Meta Platforms, Inc."


In [11]:
# Enhance exisitng dataset
df = df.merge(df_names, how='left', on='ticker')
# Checking for NA values
print('Number of records without company name:', df['company'].isna().sum())

Number of records without company name: 0


In [12]:
# Remove columns with word counts
df = df.drop(['word_count_question','word_count_answer','word_count_context','index'], axis=1)

In [13]:
df

,question,answer,context,ticker,filing,company
0,What area did NVIDIA initially focus on before...,NVIDIA initially focused on PC graphics.,"Since our original focus on PC graphics, we ha...",NVDA,2023_10K,Nvidia Corporation
1,What are some of the recent applications of GP...,Recent applications of GPU-powered deep learni...,Some of the most recent applications of GPU-po...,NVDA,2023_10K,Nvidia Corporation
2,What significant invention did NVIDIA create i...,NVIDIA invented the GPU in 1999.,Our invention of the GPU in 1999 defined moder...,NVDA,2023_10K,Nvidia Corporation
3,How does NVIDIA's platform strategy contribute...,NVIDIA's platform strategy brings together har...,"NVIDIA has a platform strategy, bringing toget...",NVDA,2023_10K,Nvidia Corporation
4,What does NVIDIA's CUDA programming model enable?,NVIDIA's CUDA programming model opened the par...,With our introduction of the CUDA programming ...,NVDA,2023_10K,Nvidia Corporation
...,...,...,...,...,...,...
6995,What was the interest rate for the 5.400% Seni...,5.400%,The 5.400% Senior Notes due in 2028 have an in...,LVS,2023_10K,Las Vegas Sands Corp.
6996,What changes were made to the LVSC Revolving C...,The Fourth Amendment to the LVSC Revolving Cre...,"On January 30, 2023, LVSC entered into amendme...",LVS,2023_10K,Las Vegas Sands Corp.
6997,What was the increase in interest expense for ...,The interest expense increased by $30 million ...,"Following the downgrades, each series of the o...",LVS,2023_10K,Las Vegas Sands Corp.
6998,What are the new leverage and interest coverag...,"As of January 2024, the new leverage ratio sho...",The amended and restated facility agreement wi...,LVS,2023_10K,Las Vegas Sands Corp.


In [ ]:
# Create the unique ids

# Create the function to generate the unique doc id
def unique_ids(doc):
    # To create a unique string to hash we take the text from different elements of the document
    combined = f"{doc['question']}-{doc['answer']}-{doc['context'][:20]}"
    # Create the hash object from the string
    hash_object = hashlib.md5(combined.encode())
    # Create a string from the hashed object
    hash_hex = hash_object.hexdigest()
    # Takne the first 8 digits of the string
    document_id = hash_hex[:8]
    return document_id

In [44]:
# Create the unique ids for each record
df['id'] = df.apply(lambda x: hashlib.md5(f"{x['question']}-{x['answer']}-{x['context']}".encode()).hexdigest()[:8], axis=1)
# Examining if we have duplicates
print('Number of duplicated ids:', sum(df['id'].duplicated()))

Number of duplicated ids: 7


In [45]:
# See the duplicates
df[df['id'].isin(df[df['id'].duplicated()].id)]

,question,answer,context,ticker,filing,company,id
2615,When does the latest expiring United States co...,2023,The latest expiring United States composition ...,JNJ,2023_10K,Johnson & Johnson,3f50113c
3860,What was the percentage increase in net premiu...,13.7%,"Net premiums written increased from $11,060 mi...",CB,2023_10K,Chubb Limited,983e4f45
3863,What was the percentage increase in net premiu...,13.7%,"Net premiums written increased from $11,060 mi...",CB,2023_10K,Chubb Limited,983e4f45
5170,What was Garmin Ltd.'s net income for the fisc...,"$1,289,636","Garmin Ltd. reported a net income of $1,289,63...",GRMN,2023_10K,Garmin Ltd.,f24a3d28
5188,What was Garmin Ltd.'s net income for the fisc...,"$1,289,636","Garmin Ltd. reported a net income of $1,289,63...",GRMN,2023_10K,Garmin Ltd.,f24a3d28
5189,What was Garmin Ltd.'s net income for the fisc...,"$1,289,636","Garmin Ltd. reported a net income of $1,289,63...",GRMN,2023_10K,Garmin Ltd.,f24a3d28
6222,Where can the Financial Statement Schedule be ...,Page S-1,The Financial Statement Schedule is located on...,IBM,2023_10K,International Business Machines Corporation,0093d68f
6231,Where can the Financial Statement Schedule be ...,Page S-1,The Financial Statement Schedule is located on...,IBM,2023_10K,International Business Machines Corporation,0093d68f
6239,Where can the Financial Statement Schedule be ...,Page S-1,The Financial Statement Schedule is located on...,IBM,2023_10K,International Business Machines Corporation,add2f3c2
6283,Where can the Financial Statement Schedule be ...,Page S-1,The Financial Statement Schedule is located on...,IBM,2023_10K,International Business Machines Corporation,add2f3c2


In [46]:
# Remove the duplicates based on id
df = df.drop_duplicates(subset=['id'], keep='first')

,question,answer,context,ticker,filing,company,id
0,What area did NVIDIA initially focus on before...,NVIDIA initially focused on PC graphics.,"Since our original focus on PC graphics, we ha...",NVDA,2023_10K,Nvidia Corporation,4f2ccc3b
1,What are some of the recent applications of GP...,Recent applications of GPU-powered deep learni...,Some of the most recent applications of GPU-po...,NVDA,2023_10K,Nvidia Corporation,ee4ed04f
2,What significant invention did NVIDIA create i...,NVIDIA invented the GPU in 1999.,Our invention of the GPU in 1999 defined moder...,NVDA,2023_10K,Nvidia Corporation,7eac6b57
3,How does NVIDIA's platform strategy contribute...,NVIDIA's platform strategy brings together har...,"NVIDIA has a platform strategy, bringing toget...",NVDA,2023_10K,Nvidia Corporation,eb49bbd0
4,What does NVIDIA's CUDA programming model enable?,NVIDIA's CUDA programming model opened the par...,With our introduction of the CUDA programming ...,NVDA,2023_10K,Nvidia Corporation,3e4c199c
...,...,...,...,...,...,...,...
6995,What was the interest rate for the 5.400% Seni...,5.400%,The 5.400% Senior Notes due in 2028 have an in...,LVS,2023_10K,Las Vegas Sands Corp.,bc9f1074
6996,What changes were made to the LVSC Revolving C...,The Fourth Amendment to the LVSC Revolving Cre...,"On January 30, 2023, LVSC entered into amendme...",LVS,2023_10K,Las Vegas Sands Corp.,2918f815
6997,What was the increase in interest expense for ...,The interest expense increased by $30 million ...,"Following the downgrades, each series of the o...",LVS,2023_10K,Las Vegas Sands Corp.,5954497e
6998,What are the new leverage and interest coverag...,"As of January 2024, the new leverage ratio sho...",The amended and restated facility agreement wi...,LVS,2023_10K,Las Vegas Sands Corp.,7c6ca115


In [48]:
# Examining how many QA records each company after cleaning
print(df.company.value_counts())

Johnson & Johnson                              199
Nvidia Corporation                             100
FedEx Corporation                              100
Caterpillar Inc.                               100
Charles Schwab Corporation                     100
                                              ... 
AbbVie Inc.                                    100
Chubb Limited                                   99
Iron Mountain Incorporated                      99
International Business Machines Corporation     98
Garmin Ltd.                                     98
Name: company, Length: 69, dtype: int64


In [50]:
# Reading the data for further use
df.to_csv('investment_data.csv', index=False)